# Getting the required libraries

In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
import json
import requests
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import wget
import re
import os

# getting the main page and links to rest all pages and compile 

Steps : <br>
1. Load the url_mail and get links to all yearwise catalogs, these catalogus have corresponding stats for that year.
2. Loop through all the pages to get all the year wise catalogs.
3. Once that is done, getting individual file name to download the data.

In [2]:
# Main page link
url_main = 'https://data.gov.in/catalogsv2?format=json&offset=0&limit=9&sort%5B_score%5D=desc&query=crime+in+india&exact_match=1'

In [9]:
path = r"./chromedriver" #chrome driver filepath
options = webdriver.ChromeOptions()
options.add_argument("download.default_directory="<specify folder to download the data>")
browser = webdriver.Chrome(executable_path=path, options=options)
browser.get(url_main)
elems = browser.find_elements_by_xpath("//a[@href]")

In [10]:
links = []
for elem in elems:
    links.append(elem.get_attribute("href")) #Getting all the links using get_attribute method

links_df = pd.DataFrame(links , columns =['link'])
links_df_get = links_df[(links_df['link'].str.contains("https://data.gov.in/catalog/crime-india-",regex = False))&~(links_df['link'].str.contains("facebook|twitter"))]
links_df_get.loc[:,'title']=links_df_get['link'].str[-16:]
links_df_get.loc[:,'sector'] = 'None'
links_df_get.columns = ['url','title','sector']

# getting urls to year-wise data files pages

In [6]:
page_links = links_df[~(links_df['link'].str.contains('filter' ,regex = False))&(links_df['link'].str.contains('crime+in+india&offset' ,regex = False))]['link'].to_list()

In [7]:
all_links = pd.DataFrame()
for pg in page_links : 
    response = requests.get(pg)
    data = json.loads(response.content)
    temp_df = json_normalize(data['records'])
    req_df = temp_df[['field_sector:name', 'title', 'url']]
    all_links = all_links.append(req_df)
    all_links = all_links.drop_duplicates('url')
    
all_links.columns = ['sector','title' ,'url'] 
final_page_links = all_links.append(links_df_get,sort=False)

# individual pages 

In [8]:
final_df_to_download = pd.DataFrame()
links_year_pages = []                                      # looping to get attributes of all the href elements

for url in final_page_links['url'].to_list():
    browser.get(url)
    print(url)
    time.sleep(np.random.uniform(3.5, 4.9))
    elems_indi = browser.find_elements_by_xpath("//a[@href]")  # parsing all the href elments 
    
    for elem_year_pages in elems_indi:
        links_year_pages.append(elem_year_pages.get_attribute("href"))
        
        #links_year_pages.append(elem_year_pages.get_attribute("href")) # not by mistake , need to think around a better way to get offset zero url
        
    links_df_year_pages = pd.DataFrame(links_year_pages , columns =['link']) # convert to dataframe to apply str.contain
    #get all pages links 
    page_links_indi = links_df_year_pages[links_df_year_pages['link'].str.contains('data.gov.in/cust-api/v1/resource' ,regex = False)]
    final_df_to_download = final_df_to_download.append(page_links_indi)
    




https://data.gov.in/catalog/crime-india-1972
https://data.gov.in/catalog/crime-india-1992
https://data.gov.in/catalog/crime-india-1987
https://data.gov.in/catalog/crime-india-2006
https://data.gov.in/catalog/crime-india-1971
https://data.gov.in/catalog/crime-india-1967
https://data.gov.in/catalog/crime-india-1961
https://data.gov.in/catalog/crime-india-1956
https://data.gov.in/catalog/crime-india-2009
https://data.gov.in/catalog/crime-india-2002
https://data.gov.in/catalog/crime-india-1991
https://data.gov.in/catalog/crime-india-1976
https://data.gov.in/catalog/crime-india-1974
https://data.gov.in/catalog/crime-india-1973
https://data.gov.in/catalog/crime-india-1999
https://data.gov.in/catalog/crime-india-1996
https://data.gov.in/catalog/crime-india-1993
https://data.gov.in/catalog/crime-india-1965
https://data.gov.in/catalog/crime-india-1953
https://data.gov.in/catalog/crime-india-1982
https://data.gov.in/catalog/crime-india-1995
https://data.gov.in/catalog/crime-india-1975
https://da

In [9]:
final_df_to_download_copy = final_df_to_download.copy(deep=True)

In [ ]:
## regex to get the offset and id for each link 

In [10]:
regex_offset = re.compile('&offset=(\d+)')
regex_id = re.compile('catalog_reference%5D=(\d+)')
final_df_to_download.loc[:,"id"] = None
final_df_to_download.loc[:,"offset"] = None

for i,j in zip(range(0,len(final_df_to_download)+1),final_df_to_download['link'].to_list()):
    final_df_to_download["offset"].values[i] = regex_offset.findall(j)
    final_df_to_download["id"].values[i] = regex_id.findall(j)


final_df_to_download["offset"] = final_df_to_download.offset.str[-1].astype('int')
final_df_to_download["id"] = final_df_to_download.id.str[-1].astype('int')
offset_max_df = final_df_to_download.groupby('id', as_index = False)['offset'].max()

In [ ]:
# Cooking all string using loop and common syntax 

In [13]:
year_link_str = []
for id_,off in zip(offset_max_df['id'].to_list(),offset_max_df['offset'].to_list()):
    for count in (np.arange(0,off+6,6)) :
        lk = "https://data.gov.in/cust-api/v1/resource/?format=json&offset="+str(count) +"&limit=6&filters%5Bfield_catalog_reference%5D="+ str(id_) +"&sort%5Bcreated%5D=desc"
        year_link_str.append(lk)
    

In [5]:
import warnings
warnings.filterwarnings('ignore')
c= 0
tmp = pd.DataFrame()
for pg in year_link_str : 
    print(pg)
    try : 
        response = requests.get(pg,verify= False)
        time.sleep(3)
        data = json.loads(response.content)
        temp_df = json_normalize(data['records'])
        tmp = tmp.append(temp_df)
    except :
        print(Exception)
        c = c + 1
        print(c)
        continue

https://data.gov.in/cust-api/v1/resource/?format=json&offset=0&limit=6&filters%5Bfield_catalog_reference%5D=381541&sort%5Bcreated%5D=desc
<class 'Exception'>
1
https://data.gov.in/cust-api/v1/resource/?format=json&offset=6&limit=6&filters%5Bfield_catalog_reference%5D=381541&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=12&limit=6&filters%5Bfield_catalog_reference%5D=381541&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=18&limit=6&filters%5Bfield_catalog_reference%5D=381541&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=24&limit=6&filters%5Bfield_catalog_reference%5D=381541&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=30&limit=6&filters%5Bfield_catalog_reference%5D=381541&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=36&limit=6&filters%5Bfield_catalog_reference%5D=381541&sort%5Bcreated%5D=desc
https:/

https://data.gov.in/cust-api/v1/resource/?format=json&offset=102&limit=6&filters%5Bfield_catalog_reference%5D=518121&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=108&limit=6&filters%5Bfield_catalog_reference%5D=518121&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=114&limit=6&filters%5Bfield_catalog_reference%5D=518121&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=120&limit=6&filters%5Bfield_catalog_reference%5D=518121&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=0&limit=6&filters%5Bfield_catalog_reference%5D=640061&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=6&limit=6&filters%5Bfield_catalog_reference%5D=640061&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=12&limit=6&filters%5Bfield_catalog_reference%5D=640061&sort%5Bcreated%5D=desc
https://data.gov.in/cust-

https://data.gov.in/cust-api/v1/resource/?format=json&offset=60&limit=6&filters%5Bfield_catalog_reference%5D=711121&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=66&limit=6&filters%5Bfield_catalog_reference%5D=711121&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=72&limit=6&filters%5Bfield_catalog_reference%5D=711121&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=0&limit=6&filters%5Bfield_catalog_reference%5D=711141&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=6&limit=6&filters%5Bfield_catalog_reference%5D=711141&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=12&limit=6&filters%5Bfield_catalog_reference%5D=711141&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=18&limit=6&filters%5Bfield_catalog_reference%5D=711141&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/

https://data.gov.in/cust-api/v1/resource/?format=json&offset=18&limit=6&filters%5Bfield_catalog_reference%5D=741681&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=24&limit=6&filters%5Bfield_catalog_reference%5D=741681&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=30&limit=6&filters%5Bfield_catalog_reference%5D=741681&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=36&limit=6&filters%5Bfield_catalog_reference%5D=741681&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=42&limit=6&filters%5Bfield_catalog_reference%5D=741681&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=48&limit=6&filters%5Bfield_catalog_reference%5D=741681&sort%5Bcreated%5D=desc
<class 'Exception'>
5
https://data.gov.in/cust-api/v1/resource/?format=json&offset=54&limit=6&filters%5Bfield_catalog_reference%5D=741681&sort%5Bcreated%5D=desc
https

https://data.gov.in/cust-api/v1/resource/?format=json&offset=42&limit=6&filters%5Bfield_catalog_reference%5D=839501&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=48&limit=6&filters%5Bfield_catalog_reference%5D=839501&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=54&limit=6&filters%5Bfield_catalog_reference%5D=839501&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=60&limit=6&filters%5Bfield_catalog_reference%5D=839501&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=0&limit=6&filters%5Bfield_catalog_reference%5D=905341&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=6&limit=6&filters%5Bfield_catalog_reference%5D=905341&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=12&limit=6&filters%5Bfield_catalog_reference%5D=905341&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/

...

https://data.gov.in/cust-api/v1/resource/?format=json&offset=42&limit=6&filters%5Bfield_catalog_reference%5D=5201421&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=48&limit=6&filters%5Bfield_catalog_reference%5D=5201421&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=54&limit=6&filters%5Bfield_catalog_reference%5D=5201421&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=60&limit=6&filters%5Bfield_catalog_reference%5D=5201421&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=66&limit=6&filters%5Bfield_catalog_reference%5D=5201421&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=72&limit=6&filters%5Bfield_catalog_reference%5D=5201421&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=78&limit=6&filters%5Bfield_catalog_reference%5D=5201421&sort%5Bcreated%5D=desc
https://data.gov.in/

https://data.gov.in/cust-api/v1/resource/?format=json&offset=48&limit=6&filters%5Bfield_catalog_reference%5D=5310221&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=54&limit=6&filters%5Bfield_catalog_reference%5D=5310221&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=60&limit=6&filters%5Bfield_catalog_reference%5D=5310221&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=66&limit=6&filters%5Bfield_catalog_reference%5D=5310221&sort%5Bcreated%5D=desc
<class 'Exception'>
19
https://data.gov.in/cust-api/v1/resource/?format=json&offset=72&limit=6&filters%5Bfield_catalog_reference%5D=5310221&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=78&limit=6&filters%5Bfield_catalog_reference%5D=5310221&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=84&limit=6&filters%5Bfield_catalog_reference%5D=5310221&sort%5Bcreated%5D=de

https://data.gov.in/cust-api/v1/resource/?format=json&offset=54&limit=6&filters%5Bfield_catalog_reference%5D=5691401&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=60&limit=6&filters%5Bfield_catalog_reference%5D=5691401&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=66&limit=6&filters%5Bfield_catalog_reference%5D=5691401&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=72&limit=6&filters%5Bfield_catalog_reference%5D=5691401&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=78&limit=6&filters%5Bfield_catalog_reference%5D=5691401&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=84&limit=6&filters%5Bfield_catalog_reference%5D=5691401&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=90&limit=6&filters%5Bfield_catalog_reference%5D=5691401&sort%5Bcreated%5D=desc
https://data.gov.in/

https://data.gov.in/cust-api/v1/resource/?format=json&offset=66&limit=6&filters%5Bfield_catalog_reference%5D=5841561&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=72&limit=6&filters%5Bfield_catalog_reference%5D=5841561&sort%5Bcreated%5D=desc
<class 'Exception'>
22
https://data.gov.in/cust-api/v1/resource/?format=json&offset=78&limit=6&filters%5Bfield_catalog_reference%5D=5841561&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=84&limit=6&filters%5Bfield_catalog_reference%5D=5841561&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=90&limit=6&filters%5Bfield_catalog_reference%5D=5841561&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=96&limit=6&filters%5Bfield_catalog_reference%5D=5841561&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=102&limit=6&filters%5Bfield_catalog_reference%5D=5841561&sort%5Bcreated%5D=d

https://data.gov.in/cust-api/v1/resource/?format=json&offset=84&limit=6&filters%5Bfield_catalog_reference%5D=5849101&sort%5Bcreated%5D=desc
<class 'Exception'>
24
https://data.gov.in/cust-api/v1/resource/?format=json&offset=90&limit=6&filters%5Bfield_catalog_reference%5D=5849101&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=96&limit=6&filters%5Bfield_catalog_reference%5D=5849101&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=102&limit=6&filters%5Bfield_catalog_reference%5D=5849101&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=108&limit=6&filters%5Bfield_catalog_reference%5D=5849101&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=114&limit=6&filters%5Bfield_catalog_reference%5D=5849101&sort%5Bcreated%5D=desc
https://data.gov.in/cust-api/v1/resource/?format=json&offset=120&limit=6&filters%5Bfield_catalog_reference%5D=5849101&sort%5Bcreated%5

In [15]:
tmp.to_csv('./all_attributes.csv')
download_links = pd.read_csv('./all_attributes.csv')

In [ ]:
# subsetting the  df columns and getting years to download year wise 

In [11]:
required_columns_each_pg = ['catalog_changed_date', 'field_datafile_url:url','published_date','title', 'type', 'url']
req_fields_parsed = download_links[required_columns_each_pg]
req_fields_parsed.loc[:,'file_name'] =req_fields_parsed['field_datafile_url:url'].apply(lambda x: str(x).split("/")[-1])


In [12]:
# getting year fro mthe file name for final folder name 
years = np.arange(1950,2020,1)
def filter(y):
    for i in years:
        if str(i) in str(y):
            return i
        else :
            continue
req_fields_parsed.loc[:,'file_name'] = req_fields_parsed['file_name'].apply(lambda x: filter(str(x)))


req_fields_parsed = req_fields_parsed[~req_fields_parsed['field_datafile_url:url'].isnull()]
req_fields_parsed['title'] = req_fields_parsed['title'].str.replace('/', '-') 

In [5]:
## final download using requests 

In [6]:
import requests
c = 1 
d = 0
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36' ,'render':'download'}

req_fields_parsed = req_fields_parsed[~req_fields_parsed.file_name.isin([2000., 1997., 1996., 1990., 1992., 1993., 1994., 1988.,
       1989., 1987., 1986., 1985., 1984., 1983., 1982., 1991., 1995.])]

for url,year,title in zip(req_fields_parsed['field_datafile_url:url'].to_list(),req_fields_parsed['file_name'].to_list(),req_fields_parsed['title'].to_list()):
    try :
        d = d+1
        if not os.path.isdir('./Desktop/crime_data/{}'.format(year)):
            os.mkdir('./Desktop/crime_data/{}'.format(year))
            time.sleep(1)
            response = requests.get(url, headers=headers)
            with open('./Desktop/crime_data/{}/{}.csv'.format(year,title), 'wb') as f:
                for chunk in response.iter_content(chunk_size = 1024): # write without using json_loads , json_normalize directly to csv
                    f.write(chunk)#
            print("Files downloaded till {}".format(d))
            
        else :
            response = requests.get(url, headers=headers)
            with open('./Desktop/crime_data/{}/{}.csv'.format(year,title), 'wb') as f:
                for chunk in response.iter_content(chunk_size = 1024):
                    f.write(chunk)
            print("Files downloaded till {}".format(d))
            time.sleep(1)
            print(url)
    except :
        print(c)
        c = c+1
        continue




Files downloaded till 1
Files downloaded till {} 2
http://data.gov.in/sites/default/files/dataurl12102015/Table_75_1999.csv
Files downloaded till {} 3
http://data.gov.in/sites/default/files/dataurl12102015/Table_76_1999.csv
Files downloaded till {} 4
http://data.gov.in/sites/default/files/dataurl12102015/Table_82_1999.csv
Files downloaded till {} 5
http://data.gov.in/sites/default/files/dataurl12102015/Table_87_1999.csv
Files downloaded till {} 6
http://data.gov.in/sites/default/files/dataurl12102015/Table_94_1999.csv
Files downloaded till {} 7
http://data.gov.in/sites/default/files/dataurl12102015/Table_97_1999.csv
Files downloaded till {} 8
http://data.gov.in/sites/default/files/dataurl12102015/Table_7_1999.csv
Files downloaded till {} 9
http://data.gov.in/sites/default/files/dataurl12102015/Table_74_1999.csv
Files downloaded till {} 10
http://data.gov.in/sites/default/files/dataurl12102015/Table_79C_1999.csv
Files downloaded till {} 11
http://data.gov.in/sites/default/files/dataurl1

Files downloaded till {} 83
http://data.gov.in/sites/default/files/dataurl12102015/Table_3.5_1999.csv
Files downloaded till {} 84
http://data.gov.in/sites/default/files/dataurl12102015/Table_3.6_1999.csv
Files downloaded till {} 85
http://data.gov.in/sites/default/files/dataurl12102015/Table_32_1999.csv
Files downloaded till {} 86
http://data.gov.in/sites/default/files/dataurl12102015/Table_34_1999.csv
Files downloaded till {} 87
http://data.gov.in/sites/default/files/dataurl12102015/Table_4_1999.csv
Files downloaded till {} 88
http://data.gov.in/sites/default/files/dataurl12102015/Table_41_1999.csv
Files downloaded till {} 89
http://data.gov.in/sites/default/files/dataurl12102015/Table_44_1999.csv
Files downloaded till {} 90
http://data.gov.in/sites/default/files/dataurl12102015/Table_46A_1999.csv
Files downloaded till {} 91
http://data.gov.in/sites/default/files/dataurl12102015/Table_6_1999.csv
Files downloaded till {} 92
http://data.gov.in/sites/default/files/dataurl12102015/Table_1

http://data.gov.in/sites/default/files/dataurl24082016/Table_20_1981.csv
Files downloaded till {} 165
http://data.gov.in/sites/default/files/dataurl24082016/Table_21_1981.csv
Files downloaded till {} 166
http://data.gov.in/sites/default/files/dataurl24082016/Table_25_1981.csv
Files downloaded till {} 167
http://data.gov.in/sites/default/files/dataurl24082016/Table_26_1981.csv
Files downloaded till {} 168
http://data.gov.in/sites/default/files/dataurl24082016/Table_32_1981.csv
Files downloaded till {} 169
http://data.gov.in/sites/default/files/dataurl24082016/Table_37_1981.csv
Files downloaded till {} 170
http://data.gov.in/sites/default/files/dataurl24082016/Table_43_1981.csv
Files downloaded till {} 171
http://data.gov.in/sites/default/files/dataurl24082016/Table_48_1981.csv
Files downloaded till {} 172
http://data.gov.in/sites/default/files/dataurl24082016/Table_1_1981.csv
Files downloaded till {} 173
http://data.gov.in/sites/default/files/dataurl24082016/Table_5_1981.csv
Files downl

Files downloaded till {} 245
http://data.gov.in/sites/default/files/dataurl02092016/Table_7A_1980.csv
Files downloaded till {} 246
http://data.gov.in/sites/default/files/dataurl02092016/Table_12_1980.csv
Files downloaded till {} 247
http://data.gov.in/sites/default/files/dataurl02092016/Table_22_1980.csv
Files downloaded till {} 248
http://data.gov.in/sites/default/files/dataurl02092016/Table_28_1980.csv
Files downloaded till {} 249
http://data.gov.in/sites/default/files/dataurl02092016/Table_38_1980.csv
Files downloaded till {} 250
http://data.gov.in/sites/default/files/dataurl02092016/Table_41_1980.csv
Files downloaded till {} 251
http://data.gov.in/sites/default/files/dataurl02092016/Table_43_1980.csv
Files downloaded till {} 252
http://data.gov.in/sites/default/files/dataurl07092016/Table_52_1979.csv
Files downloaded till {} 253
http://data.gov.in/sites/default/files/dataurl07092016/Table_53_1979.csv
Files downloaded till {} 254
http://data.gov.in/sites/default/files/dataurl0709201

http://data.gov.in/sites/default/files/dataurl07092016/Table_8.2-2015.csv
Files downloaded till {} 326
http://data.gov.in/sites/default/files/dataurl07092016/Table_8.3-2015.csv
Files downloaded till {} 327
http://data.gov.in/sites/default/files/dataurl07092016/Table_8.5-2015.csv
Files downloaded till {} 328
http://data.gov.in/sites/default/files/dataurl07092016/Table_10.5-2015.csv
Files downloaded till {} 329
http://data.gov.in/sites/default/files/dataurl07092016/Table_12.4-2015.csv
Files downloaded till {} 330
http://data.gov.in/sites/default/files/dataurl07092016/Table_13.2-2015.csv
Files downloaded till {} 331
http://data.gov.in/sites/default/files/dataurl07092016/Table_7.2-2015.csv
Files downloaded till {} 332
http://data.gov.in/sites/default/files/dataurl07092016/Table_7.3-2015.csv
Files downloaded till {} 333
http://data.gov.in/sites/default/files/dataurl07092016/Table_7.4-2015.csv
Files downloaded till {} 334
http://data.gov.in/sites/default/files/dataurl07092016/Table_7.12-2015

...

Files downloaded till {} 2619
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2002_Table_8.6.csv
Files downloaded till {} 2620
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2002_Table_8.7.csv
Files downloaded till {} 2621
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2002_Table_9.1.csv
Files downloaded till {} 2622
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2002_Table_1.5.csv
Files downloaded till {} 2623
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2002_Table_1.6.csv
Files downloaded till {} 2624
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2002_Table_1.14.csv
Files downloaded till {} 2625
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2002_Table_3.7.csv
Files downloaded till {} 2626
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2002_Table_4.2.csv
Files downloaded till {} 2627
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2002_Table_4.12.csv
Files downloaded 

Files downloaded till {} 2694
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2003_Table_17.5.csv
Files downloaded till {} 2695
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2003_Table_18.2.csv
Files downloaded till {} 2696
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2003_Table_17.7.csv
Files downloaded till {} 2697
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2003_Table_17.12.csv
Files downloaded till {} 2698
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2003_Table_18.4.csv
Files downloaded till {} 2699
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2003_Table_8.1.csv
Files downloaded till {} 2700
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2003_Table_8.7.csv
Files downloaded till {} 2701
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2003_Table_8.9.csv
Files downloaded till {} 2702
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2003_Table_10.10.csv
Files downl

Files downloaded till {} 2769
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2003_Table_4.7.csv
Files downloaded till {} 2770
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2003_Table_4.10.csv
Files downloaded till {} 2771
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2003_Table_4.12.csv
Files downloaded till {} 2772
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2003_Table_4.15.csv
Files downloaded till {} 2773
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2003_Table_6.10.csv
Files downloaded till {} 2774
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2003_Table_7.3.csv
Files downloaded till {} 2775
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2003_Table_7.6.csv
Files downloaded till {} 2776
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2003_Table_7.15.csv
Files downloaded till {} 2777
http://data.gov.in/sites/default/files/dataurl10092018/NCRB_2003_Table_4.17.csv
Files downloa

http://data.gov.in/sites/default/files/dataurl13092018/NCRB_2001_Table_17.csv
Files downloaded till {} 2845
http://data.gov.in/sites/default/files/dataurl13092018/NCRB_2001_Table_18.csv
Files downloaded till {} 2846
http://data.gov.in/sites/default/files/dataurl13092018/NCRB_2001_Table_20a.csv
Files downloaded till {} 2847
http://data.gov.in/sites/default/files/dataurl13092018/NCRB_2001_Table_25a.csv
Files downloaded till {} 2848
http://data.gov.in/sites/default/files/dataurl13092018/NCRB_2001_Table_26.csv
Files downloaded till {} 2849
http://data.gov.in/sites/default/files/dataurl13092018/NCRB_2001_Table_16a.csv
Files downloaded till {} 2850
http://data.gov.in/sites/default/files/dataurl13092018/NCRB_2001_Table_25.csv
Files downloaded till {} 2851
http://data.gov.in/sites/default/files/dataurl13092018/NCRB_2001_Table_27.csv
Files downloaded till {} 2852
http://data.gov.in/sites/default/files/dataurl13092018/NCRB_2001_Table_92.csv
Files downloaded till {} 2853
http://data.gov.in/sites/

Files downloaded till {} 2920
http://data.gov.in/sites/default/files/dataurl13092018/NCRB_2001_Table_46.csv
Files downloaded till {} 2921
http://data.gov.in/sites/default/files/dataurl13092018/NCRB_2001_Table_46a.csv
Files downloaded till {} 2922
http://data.gov.in/sites/default/files/dataurl13092018/NCRB_2001_Table_30g.csv
Files downloaded till {} 2923
http://data.gov.in/sites/default/files/dataurl13092018/NCRB_2001_Table_32.csv
Files downloaded till {} 2924
http://data.gov.in/sites/default/files/dataurl13092018/NCRB_2001_Table_32d.csv
Files downloaded till {} 2925
http://data.gov.in/sites/default/files/dataurl13092018/NCRB_2001_Table_34b.csv
Files downloaded till {} 2926
http://data.gov.in/sites/default/files/dataurl13092018/NCRB_2001_Table_34c.csv
Files downloaded till {} 2927
http://data.gov.in/sites/default/files/dataurl13092018/NCRB_2001_Table_34g.csv
Files downloaded till {} 2928
http://data.gov.in/sites/default/files/dataurl13092018/NCRB_2001_Table_34i.csv
Files downloaded till